In [1]:
import os

In [2]:
%pwd

'/Users/jay/Desktop/mlproject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/jay/Desktop/mlproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from customCNNClassifier.constants import *
from customCNNClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, param_file_path=PARAMS_FILE_PATH) -> None:
        print(read_yaml(config_file_path))
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
import gdown
from customCNNClassifier import logger
from customCNNClassifier.utils.common import get_size

In [26]:
class DataIngestion:
    def __init__(self, config=DataIngestionConfig) -> None:
        self.config = config

    def download_file(self) -> None:
        """
        Fetch data from Data Source. Data source is a constant path set in config folder.
        """
        try:
            dataset_url = self.config.source_url
            zip_download_file = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_file}.")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_file)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_file}")
        except Exception as e:
            raise e
        
    def unzip_zip_file(self):
        """
        Extracts contents of the ZIP file into the directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zipFile:
            zipFile.extractall(unzip_path)
            


In [28]:
# create pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_zip_file()

except Exception as e:
    raise e

[2024-07-26 18:38:23,569: INFO: common: Configuations from yaml file loaded]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-07-26 18:38:23,574: INFO: common: Configuations from yaml file loaded]
[2024-07-26 18:38:23,577: INFO: common: Configuations from yaml file loaded]
[2024-07-26 18:38:23,578: INFO: common: Created directory at artifacts]
[2024-07-26 18:38:23,580: INFO: common: Created directory at artifacts/data_ingestion]
[2024-07-26 18:38:23,581: INFO: 4273607534: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip.]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=4ae78804-4b5f-4089-a9be-4b1702d358d1
To: /Users/jay/Desktop/mlproject/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:05<00:00, 11.4MB/s]

[2024-07-26 18:38:32,826: INFO: 4273607534: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
